In [0]:
import cv2
import numpy as np
import os
from keras.utils import to_categorical
from tqdm import tqdm

directory = '/content/ML_Datasets/UCF_11'

classes = {
    'basketball' : 0,
    'biking' : 1,
    'diving' : 2,
    'golf_swing' : 3,
    'horse_riding' : 4,
    'soccer_juggling' : 5,
    'swing' : 6,
    'tennis_swing' : 7,
    'trampoline_jumping' : 8,
    'volleyball_spiking' : 9,
    'walking' : 10
    }

Y = []
X = []
for c in os.listdir(directory):
    print(c)
    if c != 'readme.txt':
        label = classes[c]
        d = directory + '/'+c
        for x in os.listdir(d):
            if x != 'Annotation':
                z = d+'/'+x
                for file in os.listdir(z):
                    cap = cv2.VideoCapture(z+'/'+file)
                    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
                    frames = []
                    i = 0
                    while(cap.isOpened()):
                        ret,fr = cap.read()
                        if ret!=True:
                            break

                        fr = cv2.cvtColor(fr,cv2.COLOR_BGR2GRAY)
                        fr = cv2.resize(fr,(64,64),interpolation = cv2.INTER_LINEAR)
                        fr = fgbg.apply(fr)

                        if i%3 == 0:
                            frames.append(np.array(fr))

                        i += 1
                    if len(frames) == 0:
                        print(c,x,file)
                    if len(frames) != 0:
                        X.append(np.array(frames))
                        Y.append(np.array(label))



X = np.array(X)
Y = np.array(Y)


print(X.shape)
print(Y.shape)
train_X = []
for i in range(X.shape[0]):
    print(X[i].shape)
    if X[i].shape[0] < 50:
        temp = np.zeros((50,64,64))
        temp[0:X[i].shape[0],:,:] = X[i][0]
        
    else:
        temp = X[i][0:50,:,:]

    train_X.append(temp)

train_X = np.array(train_X)
print(train_X.shape)
Y = np.reshape(Y,(1652,1))
Y = to_categorical(Y)
print(Y.shape)


np.save('X.npy',train_X)
np.save('Y.npy',Y)
 
   
    


In [0]:
import keras
from keras.layers import Dense,Dropout,Conv3D,Flatten,MaxPooling3D,Input,Add,Reshape
import numpy as np
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Model


n_classes = 11

X = np.load('X.npy')
Y = np.load('Y.npy')
X = np.reshape(X,(-1,50,64,64,1))

train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size=0.2,random_state=0)
train_X = np.concatenate((train_X,test_X))
train_Y = np.concatenate((train_Y,test_Y))
input_shape = (50,64,64,1)

inp = Input(shape=input_shape)
encode = Conv3D(64,(3,3,3),activation='relu')(inp)
encode = MaxPooling3D((2,2,2),strides = (3,2,2))(encode)
print(encode.shape)
encode = Dropout(0.4)(encode)
encode = Conv3D(128,(5,5,5),activation = 'relu')(encode)
print(encode.shape)
encode = Conv3D(64,(3,3,3),activation='relu')(encode)
encode = Dropout(0.4)(encode)
print(encode.shape)
encode = Conv3D(128,(5,5,5),activation='relu')(encode)
encode = Dropout(0.4)(encode)
print(encode.shape)

encode = Conv3D(256,(6,6,6),activation = 'relu')(encode)
print(encode.shape)

encoder = Model(inp,encode)


encode = Flatten()(encode)
print(encode.shape)


x = Dense(128,activation='relu')(encode)
x = Dropout(0.5)(x)
x = Dense(64,activation = 'relu')(x)
x = Dropout(0.4)(x)
x = Dense(n_classes,activation= 'softmax')(x)

model = Model(inputs = inp,outputs = x)
model.summary()
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.fit(train_X,train_Y,epochs = 10,validation_data=(test_X,test_Y))

model.save('3DCNN_UCF11')
encoding = encoder.predict(X)
print(encoding.shape)
encoding = np.reshape(encoding,(-1,256,256))
encoder.save('encoding')





In [0]:
from keras.models import load_model

import numpy as np

encoder = load_model('encoding')
X = np.load('X.npy')
X = np.reshape(X,(-1,50,64,64,1))
encode_X = encoder.predict(X)
np.save('encode_X.npy',encode_X)

In [0]:
import numpy as np
import keras
from keras.layers import Dense,Dropout,Conv3D,Flatten,MaxPooling3D,Input,Add,Reshape
from keras import initializers
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Model,load_model
from keras.engine.topology import Layer
import tensorflow as tf

n_classes = 11

class HierarchicalAttentionNetwork(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(HierarchicalAttentionNetwork, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim,)))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(HierarchicalAttentionNetwork, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))

        ait = K.exp(K.squeeze(K.dot(uit, self.u), -1))

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        weighted_input = x * K.expand_dims(ait)
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

X = np.load('encode_X.npy')
Y = np.load('Y.npy')
X = np.reshape(X,(-1,256,16*16))


train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size=0.2,random_state=0)
train_X = np.concatenate((train_X,test_X))
train_Y = np.concatenate((train_Y,test_Y))
input_shape = (256,16*16)



inp = Input(shape=input_shape)
encode = HierarchicalAttentionNetwork(256)(inp)
print(encode.shape)
encode = Reshape((16,16))(encode)
encode = LSTM(64,return_sequences = True,dropout = 0.2)(encode)
encode = LSTM(128,return_sequences = True,dropout = 0.4)(encode)
print(encode.shape)
encode = HierarchicalAttentionNetwork(128)(encode)
print(encode.shape)
encode = Reshape((16,8))(encode)
encode = LSTM(64,return_sequences = True)(encode)
encode = HierarchicalAttentionNetwork(64)(encode)
print(encode.shape)

x = Dense(128,activation='relu')(encode)
x = Dropout(0.5)(x)
x = Dense(64,activation = 'relu')(x)
x = Dropout(0.4)(x)
x = Dense(n_classes,activation= 'softmax')(x)

model = Model(inputs = inp,outputs = x)
model.summary()
model.compile(loss = 'categorical_crossentropy',optimizer = 'rmsprop',metrics = ['accuracy'])
model.fit(train_X,train_Y,batch_size = 64,epochs = 20,validation_data=(test_X,test_Y))
